In [ ]:
import pandas as pd
import wikipedia as wp
from pygeocoder import Geocoder
import time
from googletrans import Translator
# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from datetime import datetime
from datetime import date, timedelta

import tweepy; import json
from apikeys import twitterAccessToken as access_token
from apikeys import twitterAccessTokenSecret as access_token_secret
from apikeys import twitterConsumerKey as consumer_key
from apikeys import twitterConsumerSecretKey as consumer_secret

def parse_url( url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    listylist=[]
    for table in soup.find_all('table'):
        listylist.append(parse_html_table(table))
    return listylist

def parse_html_table( table):
    n_columns = 0; n_rows=0; column_names = []

    # Find number of rows and columns
    # we also find the column titles if we can
    for row in table.find_all('tr'):

        # Determine the number of rows in the table
        td_tags = row.find_all('td')
        if len(td_tags) > 0:
            n_rows+=1
            if n_columns == 0:
                # Set the number of columns for our table
                n_columns = len(td_tags)

        # Handle column names if we find them
        th_tags = row.find_all('th') 
        if len(th_tags) > 0 and len(column_names) == 0:
            for th in th_tags:
                column_names.append(th.get_text())

    # Safeguard on Column Titles
    if len(column_names) > 0 and len(column_names) != n_columns:
        raise Exception("Column titles do not match the number of columns")

    columns = column_names if len(column_names) > 0 else range(0,n_columns)
    df = pd.DataFrame(columns = columns,
                      index= range(0,n_rows))
    row_marker = 0
    for row in table.find_all('tr'):
        column_marker = 0
        columns = row.find_all('td')
        for column in columns:
            df.iat[row_marker,column_marker] = column.get_text()
            column_marker += 1
        if len(columns) > 0:
            row_marker += 1

    # Convert to float if possible
    for col in df:
        try:
            df[col] = df[col].astype(float)
        except ValueError:
            pass

    return df

def getCountryLanguages():
    #TODO: Use the .apply to just change the table to one dialect. Imrpove language scope later.
    df = parse_url('https://www.infoplease.com/world/countries-world/languages-spoken-each-country-world')
    countryLanguages = df[0].rename(columns={0:'country',1:'language'}).set_index('country')
    countryLanguages['language'] = [re.sub('\d+|%|\(.*\)|\s','',i).split(',')[0].split(';')[0] for i in countryLanguages['language']]
    return countryLanguages


#returns hashtag, followers, following, text, geo, date
#cityCountry example: 'paris,france'
def SearchForData(search_term, nTweets, cityCountry='',radius=100):
    
    # Setup Tweepy API Authentication
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())
    
    #--- Calculate geocordinates from cityCountry --- --- ---- ---- --- --- --- --
    geoConvertTries = 0
    while True:
        try:
            result = Geocoder.geocode(cityCountry)
        except Exception as error:
            #print('errrooooorrrrr: ',error.message)
            if 'OVER_QUERY_LIMIT' in str(error):
                print('Encountered an error:{0}\nWaiting 30 seconds and trying again.'.format(error))
                time.sleep(30)
                if geoConvertTries>10:
                    print("Could not convert geo. returning empty list")
                    return []
            elif not (re.search('^\w+,\w+$',cityCountry)):
                print("cityCountry input format is incorrect. It should be \'city,Country\' like \'paris,france\'")
                return []
            else:
                print("Could not convert geo. returning empty list")
                return []
        else:
            break
        geoConvertTries+=1
    # 34.0934,56.134,50mi
    coords = str(result[0].coordinates).replace('(','').replace(')','') + f',{radius}mi'
    coords=coords.replace(' ','')
    print(cityCountry, ": ", coords)
    #--- ---- ----- ---- ---- ---- ---- ---- --- ---- ---- --- ---- --- ---- --- --

    #--- grab tweets --- ---- ---- ---- ---- ---- ---- ---- --- --- ---- ---- ----
    maxTweets = 10000; oldest_tweet = None; unique_ids = []; desiredTweets = [];nTweetsPerDay=nTweets/8
    for day,num in zip([str(date.today() - timedelta(i)).split()[0] for i in range(8)], range(1,9)):
        tweetsPerDay=[]
        while len(tweetsPerDay) < min(nTweetsPerDay,maxTweets/8):
            #--- determine whether to grab tweets by geo or not --- ---- --- ----- --
            while True:
                try:
                    if cityCountry:
                        tweetsPerDay = api.search(search_term, count=nTweetsPerDay, result_type="recent", max_id=oldest_tweet, geocode=coords, until=day)
                    else:
                        tweetsPerDay = api.search(search_term, count=nTweetsPerDay, result_type="recent", max_id=oldest_tweet, until=day)
                except Exception as error:
                    print(error,'Trying again after 1 minute.')
                    time.sleep(60)
                else:
                    break
            #---- ----- ----- ---- ----- ---- ----- ---- ----- ---- ---- ---- ---- --

            #--- Dont go through an infinite loop trying to fill tweets that don't exist -----
            if len(tweetsPerDay['statuses'])==0:
                print(f'No tweets returned while searching for \'{search_term}\'\n',len(desiredTweets)\
                     ,'\n',day)
                return pd.DataFrame(desiredTweets)

            #--- Append relevent tweets to output listy list ---- --- ---- ---- ---- --- ---
            for tweet in tweetsPerDay['statuses']:
                # Append tweet_id to ids list if it doesn't already exist. This allows checking for duplicate tweets
                if tweet["id"] not in unique_ids :
                    unique_ids.append(tweet['id'])
                    desiredTweets.append({'text':tweet['text'], 'vader':analyzer.polarity_scores(tweet['text'])['compound'],
                                          'location':cityCountry,
                                          'hashtags':tweet['entities']['hashtags'], 'followers':tweet['user']['followers_count'],
                                         'friends_count':tweet['user']['friends_count'],'statuses_count':tweet['user']['statuses_count'],
                                          'created_at':datetime.strptime(tweet['created_at'],'%a %b %d %H:%M:%S %z %Y')})
                
                # Reassign the the oldest tweet (i.e. the max_id) subtract 1 so the previous oldest isn't included
                oldest_tweet = tweet["id"] - 1
            

    #--- Print sample tweet --- --- ---- ---- --- ---- ---- --- ---- ---- ---
    translator = Translator()
    try:
        print ('Sample Tweet:',translator.translate(desiredTweets[0]['text'], dest='en').text)
    except:
        print('there was an error translating sample tweet: ',desiredTweets[0]['text'])
    return pd.DataFrame(desiredTweets)


def GetTweetsByPopularCities(search_term, numTweets, translateToLocalLanguage = True):
    #-- Get the most populated cities from wikipedia (Thank you wikipedia library!) --
    html = wp.page("List_of_cities_by_population_density").html().encode("UTF-8")
    worldCities = pd.read_html(html)[1]
    worldCities = worldCities.drop([2,3,4],axis=1)
    worldCities = worldCities.rename(columns={0:'city',1:'population',5:'density',6:'country'})
    worldCities = worldCities.iloc[1:]
    worldCities['population'] = [int(city.split('\xa0')[-1].split('[')[0].replace(',','')) for city in worldCities['population']]
    worldCities['density'] = [int(city.split('\xa0')[-1].split('[')[0].replace(',','')) for city in worldCities['density']]
    #--- ---- ----- ----- ---- ---- ----- ---- ---- ----- --- ---- ---- ----- ---- ---
    
    #--- population per cities in United States --- ---- ---- --- ---- --- --- --- ---
    html = wp.page("List_of_United_States_cities_by_population_density").html().encode("UTF-8")
    UScities = pd.read_html(html)[1]
    UScities = UScities.drop([0,2,4,6,8],axis=1)
    UScities = UScities.rename(columns={1:'city',3:'state',5: 'land area (mi^2)',7:'density'})
    UScities = UScities.iloc[1:]
    #df['population']=[int(city.split('\xa0')[-1].split('[')[0].replace(',','')) for city in df['population']]
    UScities['density'] = [float(city.split('\xa0')[-1].split('[')[0].replace(',','')) for city in UScities['density']]
    UScities['land area (mi^2)']=[float(area.split('\xa0')[-1]) for area in UScities['land area (mi^2)']]
    #--- ---- ----- ----- ---- ---- ----- ---- ---- ----- --- ---- ---- ----- ---- ---
    
    #--- Get tweets by Worlds most densily populated cities ---- --- ---- ---- --- ---
    translator = Translator()
    comparisons=pd.DataFrame(columns=['time density','sentiment'])
    cityCount = 3
    for index,row in worldCities.iterrows():
        #-- location --- ----- --- ----
        city,pop,density,country = row
        cityCountry = city+' , '+country

        #-- language conversion --- ---- --
        languagesDf = getCountryLanguages()
        if translateToLocalLanguage:
            try:
                translatedSearch = translator.translate(search_term, src='en', dest=languagesDf.loc[country,'language']).text
            except ValueError:
                print("could not translate ", languagesDf.loc[country,'language'])
                translatedSearch=search_term
                print('translated word: ',translatedSearch)
        else:
            translatedSearch=search_term
        #--- --- --- ---- ---- --- --- ---
        
        try:
            tweetsWorld = pd.concat([tweetsWorld, SearchForData(translatedSearch, numTweets, cityCountry, 100)], axis=0)
        except:
            tweetsWorld = SearchForData(translatedSearch, numTweets, cityCountry, 100)
        print('\n')
        time.sleep(4)
        #if cityCount==0:
        #    break
        #else:
        #    cityCount-=1
    
    #--- Add US Cities --- ---- ---- ---- ---- ---
    cityCount = 5
    for index,row in UScities.iterrows():
        #-- location --- ----- --- ----
        city,state,area,density = row
        cityCountry = state+' , '+city

        try:
            tweetsUS = pd.concat([tweetsUS, SearchForData(translatedSearch, numTweets, cityCountry, max(area,5))], axis=0)
        except:
            tweetsUS = SearchForData(translatedSearch, numTweets, cityCountry, max(area,5))
        
        print('\n')
        time.sleep(4)
        #if cityCount==0:
        #    break
        #else:
        #    cityCount-=1
    return tweetsUS, tweetsWorld



tweetsUS, tweetsWorld = GetTweetsByPopularCities('trump', 200, False)
print(tweetsUS.groupby('location').mean())
print(tweetsWorld.groupby('location').mean())


                                                               #locations are not required inputs
#tweets = SearchForData(search_term='baguettes', nTweets=100, cityCountry='paris,france',radius=100)
#tweets

Manila , Philippines :  14.5995124,120.9842195,100mi
[{'message': 'Rate limit exceeded', 'code': 88}] Trying again after 1 minute.


In [124]:
tweets

,created_at,followers,friends_count,hashtags,location,statuses_count,text
0,2018-06-29 23:17:03+00:00,683,2054,[],"Manila , Philippines",33583,I have a strong feeling ....... that due to t...
1,2018-06-29 22:47:07+00:00,196,1740,[],"Manila , Philippines",5359,"@tedcruz It must be a cold day in hell, becaus..."
2,2018-06-29 22:17:22+00:00,679,111,[],"Manila , Philippines",22461,Loads of respect for @AndrewCMcCarthy - but if...
3,2018-06-29 21:49:06+00:00,1291,2499,"[{'text': 'TuckFrump', 'indices': [66, 76]}]","Manila , Philippines",62316,@BishTrumpsCray @zeitgeistbabe @IvankaTrump @S...
4,2018-06-29 21:41:01+00:00,76,108,[],"Manila , Philippines",5491,The media no longer has any credibility with p...
5,2018-06-29 21:38:04+00:00,1407,789,[],"Manila , Philippines",14301,Would you rather have Donald Trump or Kanye We...
6,2018-06-29 21:21:40+00:00,1371,1069,[],"Manila , Philippines",100819,Did Poland eventually lose World War II becaus...
7,2018-06-29 21:16:25+00:00,1464,2907,[],"Manila , Philippines",5879,"RT @Starshadow: @dimobey @ananavarro ""He was j..."
8,2018-06-29 21:11:44+00:00,49,135,"[{'text': 'StutteringJohn', 'indices': [88, 10...","Manila , Philippines",583,Hopefully the Donald has a sense of humor abou...
9,2018-06-29 20:56:39+00:00,1578,1004,[],"Manila , Philippines",35862,@flotus isn't it true when Melania Trump first...


In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
timeBetween=[]
prevCreatedAt = datetime.strptime(tweets[0]['created_at'],'%a %b %d %H:%M:%S %z %Y')
for tweet in tweets[1:]:
    #print(tweet['created_at'])
    timeBetween.append((prevCreatedAt - datetime.strptime(tweet['created_at'],'%a %b %d %H:%M:%S %z %Y')).total_seconds()/60)
    prevCreatedAt = datetime.strptime(tweet['created_at'],'%a %b %d %H:%M:%S %z %Y')
plt.plot(range(999),timeBetween)
plt.show()
plt.plot([datetime.strptime(tweet['created_at'],'%a %b %d %H:%M:%S %z %Y') for tweet in tweets][::-1],range(1000))
#plt.x('date',rotation='vertical')
plt.show()
print((datetime.strptime(tweets[-1]['created_at'],'%a %b %d %H:%M:%S %z %Y')-datetime.strptime(tweets[0]['created_at'],'%a %b %d %H:%M:%S %z %Y')).total_seconds()//60//60)

In [ ]:
#This code came from the following link:
#https://srome.github.io/Parsing-HTML-Tables-in-Python-with-BeautifulSoup-and-pandas/
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

class HTMLTableParser():

    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        listylist=[]
        for table in soup.find_all('table'):
            listylist.append(self.parse_html_table(table))
        return listylist

    def parse_html_table(self, table):
        n_columns = 0; n_rows=0; column_names = []

        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):

            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)

            # Handle column names if we find them
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())

        # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")

        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                          index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1

        # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass

        return df

#TODO: Use the .apply to just change the table to one dialect. Imrpove language scope later.
obj = HTMLTableParser()
df = obj.parse_url('https://www.infoplease.com/world/countries-world/languages-spoken-each-country-world')
countryLanguages = df[0].rename(columns={0:'country',1:'language'}).set_index('country')
newDict = []
for index, row in countryLanguages.iterrows():
    language = re.sub('\d+|%|\(.*\)|\s','',countryLanguages.loc[index].values[0]).split(',')[0].split(';')[0]
    newDict.append([index, language]) #print(index,": ",language)
newDict

newDf = pd.DataFrame(newDict)
newDf = newDf.rename(columns={0:'country',1:'language'}).set_index('country')
newDf

In [18]:
df = parse_url('https://www.infoplease.com/world/countries-world/languages-spoken-each-country-world')
countryLanguages = df[0].rename(columns={0:'country',1:'language'}).set_index('country')
countryLanguages['languages'] = [re.sub('\d+|%|\(.*\)|\s','',i).split(',')[0].split(';')[0] for i in countryLanguages['language']]
countryLanguages    

,language,languages
country,,
Afghanistan,"Dari Persian, Pashtu (both official), other Tu...",DariPersian
Albania,"Albanian (Tosk is the official dialect), Greek",Albanian
Algeria,"Arabic (official), French, Berber dialects",Arabic
Andorra,"Catalán (official), French, Castilian, Portuguese",Catalán
Angola,"Portuguese (official), Bantu and other African...",Portuguese
Antigua and Barbuda,"English (official), local dialects",English
Argentina,"Spanish (official), English, Italian, German, ...",Spanish
Armenia,"Armenian 98%, Yezidi, Russian",Armenian
Australia,"English 79%, native and other languages",English


In [ ]:
translator=Translator()
translator.translate('Hola me llamo Jennifer 😜😜', dest='en').text

In [37]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())
tweets = api.search('dalailama')
tweets['statuses'][0]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sat Jun 30 19:33:55 +0000 2018',
 'entities': {'hashtags': [{'indices': [87, 93], 'text': 'Tibet'},
   {'indices': [94, 103], 'text': 'Tibetans'},
   {'indices': [104, 114], 'text': 'FreeTibet'}],
  'symbols': [],
  'urls': [{'display_url': 'shar.es/anBniE',
    'expanded_url': 'https://shar.es/anBniE',
    'indices': [47, 70],
    'url': 'https://t.co/zAKFBK5Mna'},
   {'display_url': 'twitter.com/i/web/status/1…',
    'expanded_url': 'https://twitter.com/i/web/status/1013143614623440896',
    'indices': [116, 139],
    'url': 'https://t.co/9GU9LWHwGp'}],
  'user_mentions': [{'id': 23065876,
    'id_str': '23065876',
    'indices': [75, 85],
    'name': 'Tsem Tulku Rinpoche',
    'screen_name': 'tsemtulku'}]},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 1013143614623440896,
 'id_str': '1013143614623440896',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None

In [34]:
import pandas as pd
import wikipedia as wp
from pygeocoder import Geocoder
import time
import re
    
#-- Get the most populated cities from wikipedia (Thank you wikipedia library!) --
html = wp.page("List_of_cities_by_population_density").html().encode("UTF-8")
df = pd.read_html(html)[1]
df=df.drop([2,3,4],axis=1)
df=df.rename(columns={0:'city',1:'population',5:'density',6:'country'})
df=df.iloc[1:]
df['population']=[int(city.split('\xa0')[-1].split('[')[0].replace(',','')) for city in df['population']]
df['density']=[int(city.split('\xa0')[-1].split('[')[0].replace(',','')) for city in df['density']]
df
#--- ---- ----- ----- ---- ---- ----- ---- ---- ----- --- ---- ---- ----- ---- ---

,city,population,density,country
1,Manila,1780148,107561,Philippines
2,Ebeye,15000,107143,Marshall Islands
3,Pateros (Municipality),64147,79114,Philippines
4,Mumbai,12478447,73837,India
5,Dhaka,8523137,73583,Bangladesh
6,Bnei Brak,200162,73159,Israel
7,Caloocan,1489040,72302,Philippines
8,Levallois-Perret,63436,68458,France
9,Le PrÃ©-Saint-Gervais,18121,67047,France
10,Chennai,4681087,66961,India


In [30]:
[int(city.split('\xa0')[-1].split('[')[0].replace(',','')) for city in df['population']]

[1780148,
 15000,
 64147,
 12478447,
 8523137,
 200162,
 1489040,
 63436,
 18121,
 4681087,
 48689,
 22627,
 291972,
 4486679,
 27548,
 1183000,
 1683589,
 27084,
 48410,
 133412,
 353337,
 11481,
 669773,
 2265886,
 43756,
 643100,
 100050,
 73090,
 1072161,
 392869,
 121430,
 323257,
 252171,
 66455,
 11742120,
 529039,
 49708,
 46931,
 3075478,
 36950,
 113085,
 2936116]

In [4]:
tweets['Manila,Philippines']

,followers,following,geo,hashtags,statuses_count,text
0,154060,False,None,[],22455,@ryapee Hi Rya order ako ulit polyblender mejo...
1,83,False,None,[],3030,Yung dating saling pusa naging aso bigla // 🎶
2,643,False,None,"[{'text': 'AdoptDontShop', 'indices': [38, 52]}]",8529,sana meron din dito sa Pilipinas yung #AdoptDo...
3,2729,False,None,[],28771,"Aso nga kasi ako, bantay ako dito hahaha 😂"
4,25,False,None,[],14,lakas mangahol ng kaklase ko dinaig pa aso nam...
5,116,False,None,[],1993,@LampanoElla Dun sa aso oo HAHA
6,113,False,None,[],4495,Me: labas mo dila para lumabas dila ng aso \nS...
7,130,False,None,[],1745,cute kong aso https://t.co/twDprm7o5P
8,54,False,None,[],727,me: pabili pong dog food\ntindero: alin? ung p...
9,24,False,None,[],296,@dsgalarpez hahahaha aso ka na ba ngayon?


In [14]:


# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())
tweetsa=api.search(q='trump', geocode='52.132633,5.29126,100km')
tweetsa['statuses'][0]['user']['location']

'Delft, Holland'

In [114]:
df=pd.DataFrame({'a':1,'b':2,'c':3})
df2=pd.DataFrame({'a':1,'b':2,'c':3})
df+=df2

ValueError: If using all scalar values, you must pass an index

In [82]:
from datetime import date, timedelta
date.today() - timedelta(0)
for day in [str(date.today() - timedelta(i)).split()[0] for i in range(70)]:
    tweets=api.search('cat',geocode = '34.0934,56.134,200mi', until=day, max_id='1012848185839058950')
    print(tweets['statuses'][0]['created_at'])

Fri Jun 29 20:43:37 +0000 2018
Thu Jun 28 14:21:15 +0000 2018
Wed Jun 27 23:51:55 +0000 2018
Tue Jun 26 23:12:07 +0000 2018
Mon Jun 25 19:23:04 +0000 2018
Sun Jun 24 23:58:25 +0000 2018
Sat Jun 23 23:06:10 +0000 2018
Fri Jun 22 17:51:26 +0000 2018
Thu Jun 21 22:46:17 +0000 2018


IndexError: list index out of range

In [95]:
for a,b in zip([1,2],[2,3]):
    print(a,b)


1 2
2 3


In [122]:
#-- Get the most populated cities from wikipedia (Thank you wikipedia library!) --
html = wp.page("List_of_United_States_cities_by_population_density").html().encode("UTF-8")
df = pd.read_html(html)[1]
df=df.drop([0,1,4,5,6,8],axis=1)
df=df.rename(columns={2:'state',3:'city',7:'density'})
df=df.iloc[1:]
#df['population']=[int(city.split('\xa0')[-1].split('[')[0].replace(',','')) for city in df['population']]
df['density']=[int(city.split('\xa0')[-1].split('[')[0].replace(',','').split('.')[0]) for city in df['density']]
df
    #--- ---- ----- ----- ---- ---- ----- ---- ---- ----- --- ---- ---- ----- ---- ---

,state,city,density
1,New York City,New Jersey,57116
2,New York City,New Jersey,51810
3,New York City,New Jersey,49362
4,New York City,New Jersey,39066
5,New York City,New York,27788
6,New York City,New York,27016
7,New York City,New Jersey,24577
8,New York City,New Jersey,24060
9,Los Angeles,California,23216
10,New York City,New Jersey,22437
